# Smart E-commerce Catalog Data Analysis

## 1. Creating the Catalog, Database, and Table
   Databricks enables me to create catalogs, databases, and tables within Unity Catalog. Since I’m simulating, I’ll use Databricks SQL commands to establish a catalog, database, and a table with my specified schema.


| Column Name     | Description                                                      | Data Type            |
|------------------|------------------------------------------------------------------|----------------------|
| product_id       | Unique identifier for each product                               | Integer              |
| product_name     | Name of the product                                             | String               |
| category         | Category to which the product belongs                            | String (e.g., "Electronics", "Clothing", "Home") |
| price            | Price of the product                                            | Float                |
| stock            | Number of items available                                       | Integer              |
| rating           | Average customer rating                                          | Float (1 to 5)       |
| launch_date      | Date the product was launched                                   | Date                 |
| description      | Brief description of the product                                | String               |
| is_active        | Boolean indicating if the product is active in the catalog      | Boolean (true/false) |



   **Step 1: Create Catalog and Database**  
   First, I’ll create the catalog and database. In Databricks SQL.

In [0]:

%sql
-- Create a catalog 
--   CREATE CATALOG IF NOT EXISTS product_catalog;

-- Switch to the new catalog
--  USE CATALOG product_catalog;

/*
Note: I can't run this code because Unity Catalog is not available in the Databricks Community Edition. 
This code is just an illustration of how to create a catalog and database if Unity Catalog were enabled.
*/

**Step 2: Create the Table with the Specified Schema**  
Now, I’ll create the table in `product_db` with the structure I provided:



In [0]:
%sql
-- USE product_catalog.product_db;
CREATE DATABASE IF NOT EXISTS product_db;

CREATE TABLE IF NOT EXISTS product_table (
    product_id INT,
    product_name STRING,
    category STRING,
    price FLOAT,
    stock INT,
    rating FLOAT,
    launch_date DATE,
    description STRING,
    is_active BOOLEAN
);

**Step 3: Create the Table with the Specified Schema (Simulating Data Creation)**  

In this step, I will create a Spark DataFrame using a defined schema and generate sample data for the product catalog. The following code demonstrates the entire process, including initializing a Spark session, defining the schema, generating sample data, and introducing missing values.

In [0]:
from datetime import date
from pyspark.sql import SparkSession
from pyspark.sql import functions as F 
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, FloatType, DateType, BooleanType
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random

# Initialize a Spark session
spark = SparkSession.builder.getOrCreate()

# Define the schema
schema = StructType([
    StructField("product_id", IntegerType(), True),
    StructField("product_name", StringType(), True),
    StructField("category", StringType(), True),
    StructField("price", FloatType(), True),
    StructField("stock", IntegerType(), True),
    StructField("rating", FloatType(), True),
    StructField("launch_date", DateType(), True),
    StructField("description", StringType(), True),
    StructField("is_active", BooleanType(), True)
])
# Seed for reproducibility
random.seed(42)

# Sample size
num_products = 1000

# Generate sample data
data = [
    {
        'product_id': i,
        'product_name': f'Product {i}',
        'category': random.choice(['Electronics', 'Clothing', 'Home', 'Books', 'Toys']),
        'price': round(random.uniform(5, 500), 2),
        'stock': random.randint(0, 100),
        'rating': round(random.uniform(1, 5), 1),
        'launch_date': datetime.now() - timedelta(days=random.randint(0, 365)),
        'description': 'This is a great product.',
        'is_active': random.choice([True, False])
    }
    for i in range(1, num_products + 1)
]

# Create the DataFrame
product_df = spark.createDataFrame(data, schema=schema)

random_indices = random.sample(range(num_products), 50)

# Create a new DataFrame with missing values in the 'price' column
for idx in random_indices:
    product_df = product_df.withColumn(
        'price', 
        F.when(F.monotonically_increasing_id() == idx, None).otherwise(product_df['price'])
    )

# Write the DataFrame to the catalog table
product_df.write.mode("overwrite").saveAsTable("product_db.product_table")



**Step 4: Extracting the Table as a DataFrame**

Now, I will simulate extracting this table as a Spark DataFrame and saving it as a Pandas DataFrame.


In [0]:
# Set the catalog and database context if needed
# spark.sql("USE CATALOG product_catalog")
# spark.sql("USE product_db")

# Note: I can't run this code because Unity Catalog is not available in the Databricks Community Edition. 
# This code is just an illustration of how to create a catalog and database if Unity Catalog were enabled.


# Extract the table into a DataFrame
product_df = spark.sql("SELECT * FROM product_db.product_table")

# Save the DataFrame as a CSV file in the Databricks FileStore
product_df.write.csv('/dbfs/FileStore/catalog_data.csv', header=True, mode="overwrite")

# Convert the Spark DataFrame to a Pandas DataFrame
product_df_pandas = product_df.toPandas()

# Display the DataFrame
product_df.show()

+----------+------------+-----------+------+-----+------+-----------+--------------------+---------+
|product_id|product_name|   category| price|stock|rating|launch_date|         description|is_active|
+----------+------------+-----------+------+-----+------+-----------+--------------------+---------+
|         1|   Product 1|Electronics| 17.38|   35|   2.0| 2024-08-20|This is a great p...|     true|
|         2|   Product 2|       Toys| 48.03|   54|   1.1| 2024-09-13|This is a great p...|     true|
|         3|   Product 3|   Clothing|255.15|    3|   3.2| 2023-12-03|This is a great p...|    false|
|         4|   Product 4|   Clothing|227.36|   35|   4.2| 2024-10-27|This is a great p...|     true|
|         5|   Product 5|      Books|173.42|   19|   1.9| 2024-05-11|This is a great p...|     true|
|         6|   Product 6|Electronics|193.06|   45|   4.4| 2023-12-26|This is a great p...|    false|
|         7|   Product 7|Electronics|366.22|   68|   1.5| 2024-04-20|This is a great p...| 